# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 2**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 2. Application de la RI pour la liaison référentielle

Une application des méthodes de RI est la liaison référentielle. Cette tâche consiste à trouver le lien entre une portion de texte et la page Wikipédia qui le représentent le mieux. Dans ce cas d’application :
* Comme requêtes nous allons utiliser chaque mot dans une phrase (ou chaque mot important).
* Le corpus sont les pages Wikipédia. Donc, chaque page Wikipédia est un document à retrouver.
* La liste de résultats sont les candidats pour la liaison référentielle.

Considère par exemple la chaîne de caractères « ***Thomas and Mario are strikers playing in Munich*** ». Il existe trois sous-chaînes (***Thomas***, ***Mario*** et ***Munich***) ou mots importants qui sont susceptibles d'être identifiées dans une base de connaissances telle que Wikipédia. Une recherche de ces mots-clés dans les pages Wikipédia produirait de nombreux concepts candidats pour ces sous-chaînes. Donc un bon système doit retrouver comme premier résultats les liens aux pages Wikipédia : [Thomas Müller](https://en.wikipedia.org/wiki/Thomas_M%C3%BCller), [Mario Gómez](https://en.wikipedia.org/wiki/Mario_G%C3%B3mez) (ou Mario Götze) et FC [Bayern Munich](https://en.wikipedia.org/wiki/FC_Bayern_Munich). Cette tâche est particulièrement complexe et les méthodes de recherche d’information sont particulièrement utiles pour la résoudre.

> Plusieurs méthodes de liaison référentielle (Entity Linking en anglais) sont disponibles sur internet. Les plus connus sont :

>	* Spotlight → https://demo.dbpedia-spotlight.org/
>	* Babelfy → http://babelfy.org/
>	* TagMe → https://tagme.d4science.org/tagme/

Étapes à suivre pendant ce TP :
1. Liaison avec le texte
2. Les poids
3. Utilisation de termes
4. Le titre
5. L'outil


## 0. Installation de l’environnement de travail

Installation et configuration de librairies comment le TP précédent.

In [1]:
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [2]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-jjnyu35c/python-terrier_3688056058df471aac59ba237fcf7e44
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-jjnyu35c/python-terrier_3688056058df471aac59ba237fcf7e44
  Resolved https://github.com/terrier-org/pyterrier.git to commit 81f20bda483f6044bda89f717ae863e62bab42e5
  Preparing metadata (setup.py) ... done


In [3]:
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Lecture d'un index existant.

Ajoutez le code pour charger l'index créé sur **text** dans le TP 1.

In [7]:
# 1. Ajoutez votre pd_index.zip du TP1 à Google Colab

# 2. Changer NOM REPERTOIRE par le nom de votre dossier. Après exécuté la commande suivante
!unzip -o pd_index.zip 

Archive:  pd_index.zip
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.inverted.bf  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.meta.idx  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.properties  


In [13]:
# To create an IndexRef to an existing index
indexref2 = pt.IndexRef.of("./pd_index/data.properties")

# 1) Liaison avec le texte
En utilisant l’indexe du champ **text** de Wikipédia (comme dans le TP1) et la requête *« Thomas and Mario are strikers playing in Munich »* vérifiez si vous obtenez les pages Wikipédia respectives (comme le feraient Spotlight, Babelfy ou TagMe) :
* Thomas Müller
* Mario Gómez ou Mario Götze
* FC Bayern Munich

Voici la requete sur l'index mentionné :

In [14]:
pt.BatchRetrieve(indexref2, wmodel="BM25", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,30.771667,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,30.306808,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,29.796582,Thomas and Mario are strikers playing in Munich
3,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,3,28.570535,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,28.192863,Thomas and Mario are strikers playing in Munich
5,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,5,27.400834,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,25.804927,Thomas and Mario are strikers playing in Munich
7,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,7,25.073465,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,24.388908,Thomas and Mario are strikers playing in Munich
9,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,9,23.780821,Thomas and Mario are strikers playing in Munich


....

Seul Thomas mller est présent dans les résultats
Les deux autres ne font pas parties

....

# 2) Les poids

Utilisez les différentes méthodes de pondération prédéfinies sur Pyterrier (http://terrier.org/docs/v5.2/javadoc/org/terrier/matching/models/package-summary.html) pour voir s’il existe des méthodes que vous donnent de meilleurs résultats.

Choisir 5 méthodes (à vous de décider lesquels) et les comparer.

C’est-à-dire inspectez les 10 premiers résultats et vérifiez si les pages d’intérêt (Thomas Müller, Mario Gómez ou Mario Götze et FC Bayern Munich) font partie des résultats, puis les 20 premiers, et finalement les 50 premiers. Existent-ils de différences importantes ?

In [15]:
pt.BatchRetrieve(indexref2, wmodel="PL2", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,14.949048,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,14.685239,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,13.851380,Thomas and Mario are strikers playing in Munich
3,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,3,13.785998,Thomas and Mario are strikers playing in Munich
4,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,4,13.560615,Thomas and Mario are strikers playing in Munich
5,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,5,12.622423,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,12.323815,Thomas and Mario are strikers playing in Munich
7,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,7,12.071344,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,11.598371,Thomas and Mario are strikers playing in Munich
9,1,123459,575949,Tobias Kempe,https://simple.wikipedia.org/wiki/Tobias%20Kempe,9,11.100438,Thomas and Mario are strikers playing in Munich


In [16]:
pt.BatchRetrieve(indexref2, wmodel="DLH", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,16.484302,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,16.188615,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,16.054922,Thomas and Mario are strikers playing in Munich
3,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,3,15.371080,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,15.233306,Thomas and Mario are strikers playing in Munich
5,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,5,15.205449,Thomas and Mario are strikers playing in Munich
6,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,6,14.790709,Thomas and Mario are strikers playing in Munich
7,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,7,14.468759,Thomas and Mario are strikers playing in Munich
8,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,8,14.152757,Thomas and Mario are strikers playing in Munich
9,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,9,13.449981,Thomas and Mario are strikers playing in Munich


In [ ]:
pt.BatchRetrieve(indexref2, wmodel="Hiemstra_LM", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

....

ICI VOTRE RÉPONSE

....

# 3) Utilisation de termes

Une stratégie consiste à utiliser individuellement chaque terme de la requête pour voir si vous arrivez à trouver les pages d’intérêt. Un code exemple est donné ci-dessous, cependant, il n'est pas suffisant pour faire la tâche.

Proposer une modification de cette fonction basique (à vous de vous inspirer/inventer des modifications qui améliorent les résultats)

Une fois fini votre code, réalisez la même vérification de résultats que dans le point précédent.


In [ ]:
# MODIFIER CETTE FONCTION
def basic_term_search(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")


In [ ]:
# ICI VOTRE CODE POUR LA COMPARAISON
...


....

ICI VOTRE RÉPONSE

....

# 4) Le titre
Maintenant utilisez le champ ***title*** avec la requête complète et avec chaque terme. Expliquez  pourquoi les résultats sont différents.

In [ ]:
# ICI VOTRE CODE POUR CHARGER L'AUTRE INDEX
...

In [ ]:
# ICI VOTRE CODE POUR LA COMPARAISON
...


....

ICI VOTRE RÉPONSE

....

# 5) L'outil
Développez un outil/méthode/système capable de recevoir comme entrée un paragraphe de texte et comme sortie une liste ordonné de pages Wikipédia pour chaque unité lexical (token ou mot) dans le paragraphe.



---



En plus à l’information du TP, vous pouvez utiliser les ressources/idées suivants pour améliorer les résultats du système à développer :
- La combinaison de plusieurs méthodes de pondération c’est possible en utilisant les indications sur https://pyterrier.readthedocs.io/en/latest/pipeline_examples.html .
- Comme mentionné dans le TP1, il existe plusieurs type de champs. On a utilisé celui de la classe TextField. Renseignez vous sur les autres (StringField par exemple) et ses différences.
- Les distances d’édition sont largement utilisés par les moteurs de recherche. Par exemple si vous tapez « paris sain germain » sur Google il va vous suggérer la requête « paris saint germain » l’équipe de football. Vous pouvez utiliser les distances d’édition pour trouver des entités avec une orthographe différent que celle sur Wikipédia. Une de distance très performantes est la distance Levenshtein implémentée dans la classe LuceneLevenshteinDistance du paquet lucene.search.spell.
- Les moteurs de recherche peuvent être utilisés pour calculer la similarité entre les mots ou entre les entités. Par exemple, sur Google si vous tapez « orange » et « carrot » il y a plus de résultats que si vous tapez « purple » et « carrot ». Vous pouvez utiliser cette information pour décider si deux pages Wikipédia font partie d’un même sujet.
- Utilisez le fichier [anchor_text_en.ttl](http://downloads.dbpedia.org/2015-10/core-i18n/en/anchor_text_en.ttl.bz2) qui contient les anchor texts  de la Wikipédia en anglais. Le format du fichier est décrit [ici](https://iai-group.github.io/DBpedia-Entity/index_details.html). Transformez-le dans ce format au autre similaire pour faire l'indexation:

|             Frequency                 	|       Wikipedia Page        	| Anchor Text 	|
|------------------------------	|---------------	|-----------	|
|162| \<Saturn_Award_for_Best_Supporting_Actor> |"Best Supporting Actor"|
|14 |\<André_Golke>| "Golke"|
|13 |\<Charles_Pinckney_National_Historic_Site> |"Charles Pinckney National Historic Site"|
|25 |\<Thomas_Manfredini>| "Manfredini"|
|13 |<Marek_Zieńczuk> |"Marek Zieńczuk"|
|14 |\<Sigurd_Ulveseth>| "Sigurd Ulveseth"|
|104| <Pole_Position_(video_game)>| "Pole Position"|
|10| <Bang_(band)>| "Bang"|
|44| <George_Susce_(catcher)>| "George Susce"|
|11| \<Kyle_Rankin>| "Kyle Rankin"|
|…|…|…|

Donc, la premier colonne correspond au nombre de fois que la chaîne de caractères de la troisième colonne a été utilisé pour faire allusion à la page Wikipédia de la deuxième colonne. C’est-à-dire que "Best Supporting Actor" a été utilisé 162 fois dans Wikipédia pour faire allusion à la page Wikipédia https://en.wikipedia.org/wiki/Saturn_Award_for_Best_Supporting_Actor . Le fichier anchor_text_en.tll  contient les informations de la Wikipédia en anglais et pas de la Wikipédia simple, donc certain titres n’ont pas de correspondance.

In [ ]:
# ICI VOTRE CODE